<a href="https://colab.research.google.com/github/Pramuuu/Daily-tasks/blob/main/Machine_Learning_Pipe_Line.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Without pipeline usage

In [11]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [15]:
df = pd.read_csv('/content/titanic_train.csv')

In [12]:
df.sample(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
131,0,3,male,20.0,0,0,7.0500,S
65,1,3,male,NaN,1,1,15.2458,C


In [16]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [17]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [18]:
df.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,177
SibSp,0
Parch,0
Fare,0
Embarked,2


In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [20]:
X_train['Embarked']

,Embarked
331,S
733,S
382,S
704,S
813,S
...,...
106,S
270,S
860,S
435,S


In [21]:
#Applyting Imputation

si_age = SimpleImputer()  # mean value default is going to replace
si_embarked = SimpleImputer(strategy='most_frequent') # most frequent value i.e "S"

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [23]:
# one hot encoding Sex and Embarked
#  Now here I am not going to use drop ='First'
# as there will no dummy variable trap because we're using Decision trees;
# which is not a linear model so no issue.
# Here I am creating two different OHE objects because there are still missing
# values present .
# Yes we did filled up missing values but we got that in new numpy array in return
# handle_unknown is for future like if we get any values which we don't know about
# then we're going to ignore them eg(getting 'M' which is not a station)
# phew
ohe_sex = OneHotEncoder(sparse = False , handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False ,handle_unknown= 'ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [25]:
from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder with the updated parameter
ohe_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform the training data
X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train[['Embarked']])  # Fixed X_train_embarked

# Transform the test data
X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test[['Embarked']])  # Fixed X_test_embarked


In [26]:
X_train_embarked

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

In [27]:
# Remaining columns looks fine so well use them as it is

X_train_rem = X_train.drop(columns = ['Sex','Age','Embarked'])

X_test_rem = X_test.drop(columns = ['Sex','Age','Embarked'])

In [28]:
# Concatinating everything
# Remeber the order

X_train_transformed = np.concatenate((X_train_rem ,X_train_age,X_train_sex,X_train_embarked), axis =1)
X_test_transformed = np.concatenate((X_test_rem ,X_test_age,X_test_sex,X_test_embarked), axis =1)

In [29]:
# 1 col of Age, 2 Col of Sex , 3 COl of Embarked , 4 different features[Pclass , Sibsp, Parch, fare]
X_train_transformed.shape ,X_test_transformed.shape

((712, 11), (179, 11))

In [30]:
# Building the classifier
clf =DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

DecisionTreeClassifier()

In [31]:
y_pred = clf.predict(X_test_transformed)
y_pred

array([0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.776536312849162

In [33]:
# Let's say if we want to deploy this model to a website and predict
# if someone gives us the same data and we predict whether he/she lives or not
# then what??
import pickle

In [34]:
# wb -> write binary , rb-> read binary
# I have not taken the imputer one because I know I am going to get the age
# because I am going to give the age hahaha , so this is just to understand
# the gist of pipelines remember

pickle.dump(ohe_sex ,open('./ohe_sex.pkl' , 'wb'))
pickle.dump(ohe_embarked ,open('./ohe_embarked.pkl' , 'wb'))
pickle.dump(clf ,open('./clf.pkl' , 'wb'))

#####Think this part as a web page

In [35]:
ohe_sex_web = pickle.load(open('./ohe_sex.pkl' , 'rb'))
ohe_embarked_web = pickle.load(open('./ohe_embarked.pkl' , 'rb'))
clf_web = pickle.load(open('./clf.pkl' , 'rb'))

In [36]:
#  A user input
# Pclass / gender / age / SibSp / Parch/Fare /Embarked
test_input = np.array([2,'male' ,31.0 , 0 ,0 ,10.5 ,'S'] ,dtype =object).reshape(1,7)

In [37]:
test_input

array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [38]:
test_input_sex = ohe_sex_web.transform(test_input[:,1].reshape(1,1))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [39]:
test_input_sex


array([[0., 1.]])

In [40]:
test_input_embardked = ohe_embarked_web.transform(test_input[:,-1].reshape(1,1))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [41]:
test_input_embardked


array([[0., 0., 1., 0.]])

In [42]:
test_input_age = test_input[:,2].reshape(1,1)

In [43]:
test_input_age

array([[31.0]], dtype=object)

In [44]:
# Mantain the order
test_input_transformed =np.concatenate((test_input[:,[0,3,4,5]] ,test_input_age,
                                       test_input_sex,test_input_embardked) ,axis =1)

In [45]:
test_input_transformed.shape

(1, 11)

In [46]:
clf_web.predict(test_input_transformed)

array([1])

####PIPELINE WorkFlow

In [47]:
# Let's have a clean start, I am importing again just as to refer this as a
# new notebook

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.feature_selection import SelectKBest ,chi2
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder , MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [48]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [50]:
data= pd.read_csv('/content/titanic_train.csv')

#####pipeline using column transformer

In [51]:
# dropping some columns
data.drop(columns = ['PassengerId' , 'Name' ,'Ticket' , 'Cabin'],inplace = True)

In [52]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data.drop(columns=['Survived']),
                                                data['Survived'],
                                                test_size=0.2,
                                                random_state =42)

In [53]:
X_train.sample(5)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
356,1,female,22.0,0,1,55.000,S
865,2,female,42.0,0,0,13.000,S
846,3,male,NaN,8,2,69.550,S
173,3,male,21.0,0,0,7.925,S
15,2,female,55.0,0,0,16.000,S


In [54]:
# Imputation Transformer
# Using Index value helps in numpy array as we get numpy array in return
# rather than dataframe
# So creating column transformer in a way that it takes indexes rather names of columns

trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer() , [2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
] , remainder = 'passthrough')

In [56]:
# One Hot Encoding Transformer
trf2 = ColumnTransformer([
    ('ohe_sex_embarked' , OneHotEncoder(sparse_output = False , handle_unknown = 'ignore'),[1,6])
],remainder = 'passthrough')

In [57]:
# Scaling

trf3 = ColumnTransformer([
    ('scale', MinMaxScaler() , slice(0,10))
])

In [58]:
# let's not think about this , much
trf4 = SelectKBest(score_func= chi2 , k= 8)

In [59]:
trf5 = DecisionTreeClassifier()

In [60]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

In [61]:
from sklearn import set_config
set_config(display = 'diagram')

In [62]:
# Train
pipe.fit(X_train.values,y_train )

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f16073a6290>)),
                ('trf5', DecisionTreeClassifier())])

###Exploring the pipeline

In [63]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7f16073a6290>),
 'trf5': DecisionTreeClassifier()}

In [64]:
# getting the mean value
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [65]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [66]:
X_test.shape


(179, 7)

In [67]:
y_pred = pipe.predict(X_test.values)

In [68]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test ,y_pred)

0.6256983240223464

####Exploring the pipeline for production

In [70]:
pickle.dump(pipe,open('./pipe_original.pkl','wb'))

In [72]:
 pipe_prod = pickle.load(open('./pipe_original.pkl','rb'))

In [73]:
test_input_2  = np.array([2,'male',31.0 ,0,0,10.5,'S'] , dtype = object).reshape(1,7)

In [74]:
pipe_prod.predict(test_input_2)

array([0])

######So no changes required @production level; cool right just download new pipe and continue with work even if you do changes in the pipeline by adding or doing anything.
Column Transformers made our work clean and readable plus less confusion which is an indicator of good code.
There will be no problem at production and this one is a well managed and I guess well written code(hahah).
Try to work in pipelines, this will help alot in future.

###Let's Explore more

#####Cross Validation using Pipeline

In [75]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe ,X_train ,y_train ,scoring= 'accuracy').mean()

0.6391214419383433

####GridSearch using Pipeline

In [76]:
params = {
    'trf5__max_depth' : [1,2,3,4,5,None]
}

In [77]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe ,params ,cv =5 ,scoring = 'accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7f16073a6290>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [78]:
grid.best_score_

0.6391214419383433

In [79]:
grid.best_params_

{'trf5__max_depth': 2}

In [80]:
# Standard Scaling
from sklearn.preprocessing import StandardScaler
trf_ss = ColumnTransformer([
    ('standardscale' ,StandardScaler(),slice(0,10) )
])

In [81]:
# Pipeline without feature Scaling and StandardScalar
pipe_without_fs = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf_ss',trf_ss),
    ('trf5',trf5),
])

In [82]:
pipe_without_fs.fit(X_train ,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf_ss',
                 ColumnTransformer(transformers=[('standardscale',
                                                  StandardScaler(),
                                                  slice(0, 10, None))])),
                ('trf5', DecisionTreeClassifier())])

In [83]:
y_pred_ = pipe_without_fs.predict(X_test)

In [84]:
accuracy_score(y_test ,y_pred)

0.6256983240223464